Import Libraries and Load Datasets

In [2]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

# Load datasets
destinations_df = pd.read_csv("dataset/Expanded_Destinations.csv")
reviews_df = pd.read_csv("dataset/Final_Updated_Expanded_Reviews.csv")
userhistory_df = pd.read_csv("dataset/Final_Updated_Expanded_UserHistory.csv")
users_df = pd.read_csv("dataset/Final_Updated_Expanded_Users.csv")

In [3]:
destinations_df.head()

DestinationID               Name              State        Type  \
0              1          Taj Mahal      Uttar Pradesh  Historical   
1              2        Goa Beaches                Goa       Beach   
2              3        Jaipur City          Rajasthan        City   
3              4  Kerala Backwaters             Kerala      Nature   
4              5         Leh Ladakh  Jammu and Kashmir   Adventure   

   Popularity BestTimeToVisit  
0    8.691906         Nov-Feb  
1    8.605032         Nov-Mar  
2    9.225372         Oct-Mar  
3    7.977386         Sep-Mar  
4    8.399822         Apr-Jun

In [5]:
reviews_df.head()

ReviewID  DestinationID  UserID  Rating            ReviewText
0         1            178     327       2  Incredible monument!
1         2            411     783       1    Loved the beaches!
2         3            927      12       2   A historical wonder
3         4            358     959       3  Incredible monument!
4         5            989     353       2    Loved the beaches!

In [6]:
userhistory_df.head()

HistoryID  UserID  DestinationID   VisitDate  ExperienceRating
0          1     525            760  2024-01-01                 3
1          2     184            532  2024-02-15                 5
2          3     897            786  2024-03-20                 2
3          4     470            660  2024-01-01                 1
4          5     989            389  2024-02-15                 4

In [7]:
users_df.head()

UserID   Name              Email          Preferences  Gender  \
0       1  Kavya  kavya@example.com  Beaches, Historical  Female   
1       2  Rohan  rohan@example.com    Nature, Adventure    Male   
2       3  Kavya  kavya@example.com     City, Historical  Female   
3       4  Anika  anika@example.com  Beaches, Historical  Female   
4       5  Tanvi  tanvi@example.com    Nature, Adventure  Female   

   NumberOfAdults  NumberOfChildren  
0               1                 0  
1               2                 2  
2               2                 0  
3               1                 0  
4               2                 2

Data Preprocessing
We merge all datasets into a single dataset to make it ready for building recommendation models.

In [8]:
destinations_reviews = pd.merge(destinations_df, reviews_df, on='DestinationID', how='inner')
destinations_reviews

DestinationID               Name              State        Type  \
0                1          Taj Mahal      Uttar Pradesh  Historical   
1                2        Goa Beaches                Goa       Beach   
2                2        Goa Beaches                Goa       Beach   
3                4  Kerala Backwaters             Kerala      Nature   
4                5         Leh Ladakh  Jammu and Kashmir   Adventure   
..             ...                ...                ...         ...   
994            998        Jaipur City          Rajasthan        City   
995            999  Kerala Backwaters             Kerala      Nature   
996            999  Kerala Backwaters             Kerala      Nature   
997            999  Kerala Backwaters             Kerala      Nature   
998           1000         Leh Ladakh  Jammu and Kashmir   Adventure   

     Popularity BestTimeToVisit  ReviewID  UserID  Rating  \
0      8.691906         Nov-Feb       666     479       1   
1      8.605032         Nov-Mar        90     558       2   
2      8.605032         Nov-Mar       130     528       3   
3      7.977386         Sep-Mar       869     126       5   
4      8.399822         Apr-Jun       402     947       1   
..          ...             ...       ...     ...     ...   
994    8.308435         Oct-Mar       578     287       4   
995    8.961741         Sep-Mar       178     427       2   
996    8.961741         Sep-Mar       260     477       2   
997    8.961741         Sep-Mar       607     221       4   
998    8.939036         Apr-Jun       326     365       5   

               ReviewText  
0     A historical wonder  
1     A historical wonder  
2    Incredible monument!  
3      Loved the beaches!  
4     A historical wonder  
..                    ...  
994    Loved the beaches!  
995  Incredible monument!  
996    Loved the beaches!  
997  Incredible monument!  
998    Loved the beaches!  

[999 rows x 10 columns]

In [9]:
userhistory_destinations_reviews = pd.merge(userhistory_df, destinations_reviews, on='UserID', how='inner')
userhistory_destinations_reviews

HistoryID  UserID  DestinationID_x   VisitDate  ExperienceRating  \
0            2     184              532  2024-02-15                 5   
1            2     184              532  2024-02-15                 5   
2            4     470              660  2024-01-01                 1   
3            5     989              389  2024-02-15                 4   
4            5     989              389  2024-02-15                 4   
..         ...     ...              ...         ...               ...   
988        998     959              660  2024-02-15                 4   
989        998     959              660  2024-02-15                 4   
990        999     538               30  2024-03-20                 5   
991        999     538               30  2024-03-20                 5   
992        999     538               30  2024-03-20                 5   

     DestinationID_y               Name          State        Type  \
0                 61          Taj Mahal  Uttar Pradesh  Historical   
1                538        Jaipur City      Rajasthan        City   
2                102        Goa Beaches            Goa       Beach   
3                138        Jaipur City      Rajasthan        City   
4                817        Goa Beaches            Goa       Beach   
..               ...                ...            ...         ...   
988              113        Jaipur City      Rajasthan        City   
989              358        Jaipur City      Rajasthan        City   
990              207        Goa Beaches            Goa       Beach   
991              234  Kerala Backwaters         Kerala      Nature   
992              377        Goa Beaches            Goa       Beach   

     Popularity BestTimeToVisit  ReviewID  Rating            ReviewText  
0      9.255411         Nov-Feb       701       4    Loved the beaches!  
1      8.612910         Oct-Mar       399       5   A historical wonder  
2      7.901206         Nov-Mar       818       2    Loved the beaches!  
3      9.195643         Oct-Mar       447       4   A historical wonder  
4      7.999027         Nov-Mar        70       4  Incredible monument!  
..          ...             ...       ...     ...                   ...  
988    7.576535         Oct-Mar       361       5  Incredible monument!  
989    7.738761         Oct-Mar         4       3  Incredible monument!  
990    8.560557         Nov-Mar       853       3  Incredible monument!  
991    9.420961         Sep-Mar        94       1  Incredible monument!  
992    7.922286         Nov-Mar       826       1  Incredible monument!  

[993 rows x 14 columns]

In [10]:
final_dataset = pd.merge(userhistory_destinations_reviews, users_df, on='UserID', how='inner')
final_dataset

HistoryID  UserID  DestinationID_x   VisitDate  ExperienceRating  \
0            2     184              532  2024-02-15                 5   
1            2     184              532  2024-02-15                 5   
2            4     470              660  2024-01-01                 1   
3            5     989              389  2024-02-15                 4   
4            5     989              389  2024-02-15                 4   
..         ...     ...              ...         ...               ...   
988        998     959              660  2024-02-15                 4   
989        998     959              660  2024-02-15                 4   
990        999     538               30  2024-03-20                 5   
991        999     538               30  2024-03-20                 5   
992        999     538               30  2024-03-20                 5   

     DestinationID_y             Name_x          State        Type  \
0                 61          Taj Mahal  Uttar Pradesh  Historical   
1                538        Jaipur City      Rajasthan        City   
2                102        Goa Beaches            Goa       Beach   
3                138        Jaipur City      Rajasthan        City   
4                817        Goa Beaches            Goa       Beach   
..               ...                ...            ...         ...   
988              113        Jaipur City      Rajasthan        City   
989              358        Jaipur City      Rajasthan        City   
990              207        Goa Beaches            Goa       Beach   
991              234  Kerala Backwaters         Kerala      Nature   
992              377        Goa Beaches            Goa       Beach   

     Popularity BestTimeToVisit  ReviewID  Rating            ReviewText  \
0      9.255411         Nov-Feb       701       4    Loved the beaches!   
1      8.612910         Oct-Mar       399       5   A historical wonder   
2      7.901206         Nov-Mar       818       2    Loved the beaches!   
3      9.195643         Oct-Mar       447       4   A historical wonder   
4      7.999027         Nov-Mar        70       4  Incredible monument!   
..          ...             ...       ...     ...                   ...   
988    7.576535         Oct-Mar       361       5  Incredible monument!   
989    7.738761         Oct-Mar         4       3  Incredible monument!   
990    8.560557         Nov-Mar       853       3  Incredible monument!   
991    9.420961         Sep-Mar        94       1  Incredible monument!   
992    7.922286         Nov-Mar       826       1  Incredible monument!   

     Name_y               Email          Preferences  Gender  NumberOfAdults  \
0    Ishaan  ishaan@example.com  Beaches, Historical    Male               1   
1    Ishaan  ishaan@example.com  Beaches, Historical    Male               1   
2     Arjun   arjun@example.com    Nature, Adventure    Male               2   
3    Ritvik  ritvik@example.com    Nature, Adventure    Male               2   
4    Ritvik  ritvik@example.com    Nature, Adventure    Male               2   
..      ...                 ...                  ...     ...             ...   
988  Ritvik  ritvik@example.com    Nature, Adventure    Male               1   
989  Ritvik  ritvik@example.com    Nature, Adventure    Male               1   
990   Anika   anika@example.com  Beaches, Historical  Female               2   
991   Anika   anika@example.com  Beaches, Historical  Female               2   
992   Anika   anika@example.com  Beaches, Historical  Female               2   

     NumberOfChildren  
0                   2  
1                   2  
2                   1  
3                   1  
4                   1  
..                ...  
988                 1  
989                 1  
990                 1  
991                 1  
992                 1  

[993 rows x 20 columns]

In [12]:
final_dataset.shape

(993, 20)

In [13]:
final_dataset.duplicated().sum()

np.int64(0)

In [14]:
final_dataset.isnull().sum()

HistoryID           0
UserID              0
DestinationID_x     0
VisitDate           0
ExperienceRating    0
DestinationID_y     0
Name_x              0
State               0
Type                0
Popularity          0
BestTimeToVisit     0
ReviewID            0
Rating              0
ReviewText          0
Name_y              0
Email               0
Preferences         0
Gender              0
NumberOfAdults      0
NumberOfChildren    0
dtype: int64

Create a text-based feature for the destinations

In [15]:
final_dataset['Features'] = final_dataset['Type'] + ' ' + final_dataset['State'] + ' ' + final_dataset['BestTimeToVisit'] + ' ' + final_dataset['Preferences']
final_dataset

HistoryID  UserID  DestinationID_x   VisitDate  ExperienceRating  \
0            2     184              532  2024-02-15                 5   
1            2     184              532  2024-02-15                 5   
2            4     470              660  2024-01-01                 1   
3            5     989              389  2024-02-15                 4   
4            5     989              389  2024-02-15                 4   
..         ...     ...              ...         ...               ...   
988        998     959              660  2024-02-15                 4   
989        998     959              660  2024-02-15                 4   
990        999     538               30  2024-03-20                 5   
991        999     538               30  2024-03-20                 5   
992        999     538               30  2024-03-20                 5   

     DestinationID_y             Name_x          State        Type  \
0                 61          Taj Mahal  Uttar Pradesh  Historical   
1                538        Jaipur City      Rajasthan        City   
2                102        Goa Beaches            Goa       Beach   
3                138        Jaipur City      Rajasthan        City   
4                817        Goa Beaches            Goa       Beach   
..               ...                ...            ...         ...   
988              113        Jaipur City      Rajasthan        City   
989              358        Jaipur City      Rajasthan        City   
990              207        Goa Beaches            Goa       Beach   
991              234  Kerala Backwaters         Kerala      Nature   
992              377        Goa Beaches            Goa       Beach   

     Popularity  ... ReviewID  Rating            ReviewText  Name_y  \
0      9.255411  ...      701       4    Loved the beaches!  Ishaan   
1      8.612910  ...      399       5   A historical wonder  Ishaan   
2      7.901206  ...      818       2    Loved the beaches!   Arjun   
3      9.195643  ...      447       4   A historical wonder  Ritvik   
4      7.999027  ...       70       4  Incredible monument!  Ritvik   
..          ...  ...      ...     ...                   ...     ...   
988    7.576535  ...      361       5  Incredible monument!  Ritvik   
989    7.738761  ...        4       3  Incredible monument!  Ritvik   
990    8.560557  ...      853       3  Incredible monument!   Anika   
991    9.420961  ...       94       1  Incredible monument!   Anika   
992    7.922286  ...      826       1  Incredible monument!   Anika   

                  Email          Preferences  Gender NumberOfAdults  \
0    ishaan@example.com  Beaches, Historical    Male              1   
1    ishaan@example.com  Beaches, Historical    Male              1   
2     arjun@example.com    Nature, Adventure    Male              2   
3    ritvik@example.com    Nature, Adventure    Male              2   
4    ritvik@example.com    Nature, Adventure    Male              2   
..                  ...                  ...     ...            ...   
988  ritvik@example.com    Nature, Adventure    Male              1   
989  ritvik@example.com    Nature, Adventure    Male              1   
990   anika@example.com  Beaches, Historical  Female              2   
991   anika@example.com  Beaches, Historical  Female              2   
992   anika@example.com  Beaches, Historical  Female              2   

     NumberOfChildren                                           Features  
0                   2  Historical Uttar Pradesh Nov-Feb Beaches, Hist...  
1                   2         City Rajasthan Oct-Mar Beaches, Historical  
2                   1                Beach Goa Nov-Mar Nature, Adventure  
3                   1           City Rajasthan Oct-Mar Nature, Adventure  
4                   1                Beach Goa Nov-Mar Nature, Adventure  
..                ...                                                ...  
988                 1           City Rajasthan Oct-Mar N

Apply TF-IDF vectorizer to create feature vectors for destinations

In [16]:
vectorizer = TfidfVectorizer(stop_words='english')
destination_features = vectorizer.fit_transform(final_dataset['Features'])
destination_features.toarray()

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.43591168],
       [0.        , 0.        , 0.        , ..., 0.51966696, 0.        ,
        0.        ],
       [0.3295216 , 0.        , 0.52286725, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.52557847, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.5222971 ,
        0.        ],
       [0.        , 0.        , 0.52557847, ..., 0.        , 0.        ,
        0.        ]], shape=(993, 20))

Compute cosine similarity between destinations

In [17]:
cosine_sim = cosine_similarity(destination_features, destination_features)
cosine_sim

array([[1.        , 0.26008436, 0.12453959, ..., 0.37339618, 0.2568862 ,
        0.37339618],
       [0.26008436, 1.        , 0.09314204, ..., 0.32111179, 0.33233519,
        0.32111179],
       [0.12453959, 0.09314204, 1.        , ..., 0.77885997, 0.21327007,
        0.77885997],
       ...,
       [0.37339618, 0.32111179, 0.77885997, ..., 1.        , 0.3171632 ,
        1.        ],
       [0.2568862 , 0.33233519, 0.21327007, ..., 0.3171632 , 1.        ,
        0.3171632 ],
       [0.37339618, 0.32111179, 0.77885997, ..., 1.        , 0.3171632 ,
        1.        ]], shape=(993, 993))

Define recommendation function

In [37]:
def recommend_destinations(user_id, userhistory_df, destinations_df, cosine_sim):
    """
    Recommends top 5 destinations for a given user based on similarity scores.
    Args:
    - user_id: ID of the user.
    - userhistory_df: User history DataFrame containing 'UserID' and 'DestinationID'.
    - destinations_df: Destinations DataFrame containing destination details.
    - cosine_sim: Cosine similarity matrix for destinations.
    Returns:
    - DataFrame with recommended destinations and their details.
    """
    # Get the destinations the user has visited
    visited_destinations = userhistory_df[userhistory_df['UserID'] == user_id]['DestinationID'].values
    print(f'visited destinations: {visited_destinations}')
    
    # Calculate similarity scores for visited destinations
    similar_scores = np.sum(cosine_sim[visited_destinations - 1], axis=0)
    print(f'similar scores: {similar_scores}')
    
    # Recommend the top destinations the user hasn't visited yet
    recommended_destinations_idx = np.argsort(similar_scores)[::-1]
    print(f'top destinations the user has not visited: {recommended_destinations_idx}')
    
    recommendations = []
    for idx in recommended_destinations_idx:
        if destinations_df.iloc[idx]['DestinationID'] not in visited_destinations:
            # Append detailed information for each recommendation
            recommendations.append(destinations_df.iloc[idx][[
                'DestinationID', 'Name', 'State', 'Type', 'Popularity', 'BestTimeToVisit'
            ]].to_dict())
        if len(recommendations) >= 5:
            break
    
    # Convert recommendations to a DataFrame
    return pd.DataFrame(recommendations)

# Example: Recommend destinations for user with ID 184
recommended_destinations = recommend_destinations(184, userhistory_df, destinations_df, cosine_sim)
# Display recommendations
recommended_destinations

visited destinations: [532 394 878 485]
similar scores: [1.365582   1.47139449 2.00743463 1.45203665 2.00743463 1.45203665
 0.71109306 2.51312731 1.75524379 1.75524379 2.00743463 2.02843047
 1.47139449 1.11308647 1.45203665 2.00743463 1.45203665 1.47139449
 0.64137545 2.02843047 0.71109306 1.77794905 1.75524379 0.64137545
 1.365582   1.365582   1.11308647 0.56085151 0.27698856 0.27698856
 1.11308647 2.51312731 0.71109306 1.77794905 0.31114175 2.00743463
 0.56085151 2.00743463 0.27698856 1.42201092 2.00743463 0.71109306
 0.64137545 0.64137545 1.365582   1.11308647 1.77794905 2.51312731
 1.45203665 1.45203665 1.45203665 1.42201092 0.27698856 2.02843047
 2.02843047 0.71109306 1.11308647 0.71109306 1.77794905 1.11308647
 2.51312731 1.77794905 2.02843047 0.31114175 0.56085151 1.365582
 1.42201092 1.42201092 0.56085151 1.75524379 1.77794905 2.51312731
 1.365582   0.71109306 0.56085151 1.77794905 1.77794905 1.75524379
 0.31114175 1.365582   1.47139449 1.47139449 2.51312731 0.71109306
 1.11308

DestinationID         Name          State        Type  Popularity  \
0             32  Goa Beaches            Goa       Beach    7.806330   
1             72  Goa Beaches            Goa       Beach    9.498129   
2             48  Jaipur City      Rajasthan        City    9.072662   
3             61    Taj Mahal  Uttar Pradesh  Historical    9.255411   
4            198  Jaipur City      Rajasthan        City    8.255893   

  BestTimeToVisit  
0         Nov-Mar  
1         Nov-Mar  
2         Oct-Mar  
3         Nov-Feb  
4         Oct-Mar